In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils.prune as prune

from scipy import io
import os

/Users/goldenyoo/miniforge3/envs/mac_cpu/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
mat_file = io.loadmat('/Users/goldenyoo/Library/Mobile Documents/com~apple~CloudDocs/BioCAS_prepare/Python_code/Data_center/one_dx/Calib_data_1.mat')

In [ ]:
K1 = mat_file['K1']
A1 = mat_file['A1']

K2 = mat_file['K2']
A2 = mat_file['A2']

# K1 = np.swapaxes(K1,0,2)
# K1 = np.swapaxes(K1,1,2)
# A1 = np.swapaxes(A1,0,2)
# A1 = np.swapaxes(A1,1,2)

# K2 = np.swapaxes(K2,0,2)
# K2 = np.swapaxes(K2,1,2)
# A2 = np.swapaxes(A2,0,2)
# A2 = np.swapaxes(A2,1,2)


Y1 = mat_file['Y1']
Y2 = mat_file['Y2']

In [ ]:
k1 = torch.FloatTensor(K1)

In [2]:
n_class = 22
n_hidden = 16
dtype = torch.float

class TextLSTM(nn.Module):
  def __init__(self):
    super(TextLSTM, self).__init__()

    self.lstm_1 = nn.LSTM(input_size=n_class, hidden_size=n_hidden, dropout=0.3)
    self.lstm_2 = nn.LSTM(input_size=n_class, hidden_size=n_hidden, dropout=0.3)
    self.fc = nn.Linear(n_hidden*2, n_class)

  def forward(self, hidden_and_cell, X):
    X = X.transpose(0, 1)
    outputs1, (h_n1,c_n1) = self.lstm_1(X, hidden_and_cell)
    outputs2, (h_n2,c_n2) = self.lstm_2(X, hidden_and_cell)

    outputs = torch.cat((h_n1[-1],h_n2[-1]), dim=2)  # 최종 예측 Hidden Layer

    model = self.fc(outputs)  # 최종 예측 최종 출력 층
    return model

In [3]:
model = TextLSTM()
print(model)

TextLSTM(
  (lstm_1): LSTM(22, 16, dropout=0.3)
  (lstm_2): LSTM(22, 16, dropout=0.3)
  (fc): Linear(in_features=32, out_features=22, bias=True)
)


/Users/goldenyoo/miniforge3/envs/mac_cpu/lib/python3.8/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [ ]:
input = torch.zeros(1, n_hidden,n_class)
hidden = torch.zeros(1, 1, n_hidden, requires_grad=True)
cell = torch.zeros(1, 1, n_hidden, requires_grad=True)

model((hidden,cell),input)